<a href="https://colab.research.google.com/github/daC0rtes/Proceso-educativo/blob/main/fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Es un trabajo llevado a cabo con la guia de Siddhardhan (yt) no me apropio del codigo, debe ser visto como produccion academica recopilatoria, donde los derechos de autor se le atribuyen a la persona ya mencionada.  


In [2]:
import numpy as np
import pandas as pd
import re #regular expresions
from nltk.corpus  import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import  nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words('english')) #letras en las cuales los parrafos deberian parar u omitir

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-procesing

In [8]:
#cargar la data como un dataframe de pandas
news_data= pd.read_csv('/content/train.csv')

In [10]:
news_data.head() # para label, 1 es no creible, 0 es creible

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
#contar el numero de valores faltantes
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
#reemplazamos los lugares vacios con texto no escrito
news_data = news_data.fillna('')

In [13]:
# se unen las columnas autor y titulo
news_data['content']= news_data['author']+' '+ news_data['title']

In [14]:
print(news_data['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [16]:
#porque ahora nos basaremos solo en este nuevo data set para las predicciones
#separamos la data y el label
x=news_data.drop(columns='label',axis=1)
y=news_data['label']

Stemming:
proceso por el cual se reduce la o las palabras a su raiz

ejemplo: actor actriz actuacion --> act (ingles)

In [17]:
port_stem=PorterStemmer()

In [18]:
def stemming(content):
  raiz_content = re.sub('[^a-zA-Z]',' ', content)
  raiz_content = raiz_content.lower()
  raiz_content = raiz_content.split()
  raiz_content = [port_stem.stem(word) for word in raiz_content if not word in stopwords.words('english')]
  raiz_content = ' '.join(raiz_content)
  return raiz_content

In [19]:
news_data['content']= news_data['content'].apply(stemming)

In [21]:
print(news_data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [22]:
#separamos los datos  y label
x= news_data['content'].values
y= news_data['label'].values

In [23]:
#convertimos la data de tipo texto a tipo numerico
vector = TfidfVectorizer()
vector.fit(x)

x= vector.transform(x)

In [24]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Separamos los datos en aquellos que seran de prueba y los de entrenamiento

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size= 0.2, stratify=y, random_state=2 )

Entrenamos el modelo con regresion logistica

In [29]:
modelo = LogisticRegression()
modelo.fit(X_train,Y_train)

LogisticRegression()

Evaluacion y presicion

In [30]:
#precicion de los datos entrenados (train)
prediccion_x_train = modelo.predict(X_train)
data_entrenada_precision = accuracy_score(prediccion_x_train,Y_train)

In [31]:
print("puntaje de desicion de los datos entrenados:", data_entrenada_precision )

puntaje de desicion de los datos entrenados: 0.9865985576923076


In [32]:
#precicion de los datos de prueba (test)
prediccion_x_test = modelo.predict(X_test)
data_prueba_precision = accuracy_score(prediccion_x_test,Y_test)

In [33]:
print("puntaje de desicion de los datos de prueba:", data_prueba_precision )

puntaje de desicion de los datos de prueba: 0.9790865384615385


Creamos un sistema predictivo

In [38]:
x_nueva = X_test[0]

prediccion= modelo.predict(x_nueva)
print(prediccion)
if (prediccion==[0]):
  print("la noticia es verdadera")
else:
  print("la noticia es falsa")


[1]
la noticia es falsa
